In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 1e-4
PRIOR_FLAG = False
ONLY_FORWARD = True
CUDA = torch.cuda.is_available()
PATH = 'ep-onlyf-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:0')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE, PRIOR_FLAG, ONLY_FORWARD)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
if PRIOR_FLAG:
    optimizer =  torch.optim.Adam(list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
    models = (enc_eta, enc_z)
else:
    oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
    if CUDA:
        oneshot_eta.cuda().to(DEVICE)
    optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
    models = (oneshot_eta, enc_eta, enc_z)

In [4]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (121s),  symKL_DB_eta: 463.467,  symKL_DB_z: 508.143,  loss: -1726.485,  ess: 3.036,  kl_eta_ex: 1366.089,  kl_eta_in: 229.879,  kl_z_ex: 366.283,  kl_z_in: 38.474
epoch: 1\250 (124s),  symKL_DB_eta: 329.347,  symKL_DB_z: 290.235,  loss: -1372.047,  ess: 3.377,  kl_eta_ex: 876.080,  kl_eta_in: 227.244,  kl_z_ex: 203.559,  kl_z_in: 30.608
epoch: 2\250 (122s),  symKL_DB_eta: 240.840,  symKL_DB_z: 186.489,  loss: -1161.583,  ess: 3.583,  kl_eta_ex: 604.285,  kl_eta_in: 226.997,  kl_z_ex: 124.453,  kl_z_in: 25.634
epoch: 3\250 (125s),  symKL_DB_eta: 183.300,  symKL_DB_z: 135.300,  loss: -1026.757,  ess: 3.754,  kl_eta_ex: 446.365,  kl_eta_in: 231.074,  kl_z_ex: 83.220,  kl_z_in: 21.350
epoch: 4\250 (120s),  symKL_DB_eta: 149.022,  symKL_DB_z: 101.532,  loss: -944.318,  ess: 3.953,  kl_eta_ex: 352.893,  kl_eta_in: 244.535,  kl_z_ex: 58.065,  kl_z_in: 17.374
epoch: 5\250 (121s),  symKL_DB_eta: 128.644,  symKL_DB_z: 78.634,  loss: -893.210,  ess: 4.159,  kl_eta_ex: 297.952,  kl_e

In [5]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)

In [6]:
BATCH_SIZE_TEST = 50
Model_Params_Test = (N, K, D, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE_TEST)
obs, metric_step, reused = test(Eubo_cfz_os_eta, Data, Train_Params, Model_Params_Test, models)
(q_eta, _, q_z, _, _, _) = reused

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-589955aded6e>", line 3, in <module>
    obs, metric_step, reused = test(Eubo_cfz_os_eta, Data, Train_Params, Model_Params_Test, models)
NameError: name 'Eubo_cfz_os_eta' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, nu

NameError: name 'Eubo_cfz_os_eta' is not defined

In [ ]:
%time plot_samples(obs, q_eta, q_z, PATH)

In [ ]:
incremental_gap = symkl_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')

In [ ]:
incremental_gap